In [1]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.tools import FunctionTool
import os


In [2]:
#llm = Ollama(model="llama3.2", request_timeout=120.0)

llm = OpenAILike(model="meta-llama/Llama-3.3-70B-Instruct", request_timeout=120.0, api_base="https://llm.scads.ai/v1", api_key=os.environ.get('SCADSAI_API_KEY'))


In [3]:
tools = []

@tools.append
def estimate_delivery_date_of_order(order_id:str) -> str:
    """Estimate the delivery date of a package identified by its order id."""
    if order_id == "292123":
        return "Friday"
    if order_id == "292456":
        return "Thursday"
    if order_id == "292789":
        return "Saturday"
    return "Unknown"

@tools.append
def get_recent_order_id(customer_name:str) -> str:
    """Get the most recent order id for a given customer"""
    if customer_name.lower() == "robert":
        return "292123"
    if customer_name.lower() == "alice":
        return "292456"
    if customer_name.lower() == "ivy":
        return "292789"
    if customer_name.lower() == "dennis":
        return "2921010"
    return "unknown"

@tools.append
def get_weather(city:str)->str:
    """Returns the current weather in a given city."""
    return "sunny"

In [4]:
agent = ReActAgent.from_tools([FunctionTool.from_defaults(fn=t) for t in tools], llm=llm, verbose=True)

response = agent.chat("My Name is Robert. I ordered something. And I would like to know when it will arrive.")
response.response

> Running step 034c2a59-1d4e-4388-9757-0331f32b0747. Step input: My Name is Robert. I ordered something. And I would like to know when it will arrive.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_recent_order_id
Action Input: {'customer_name': 'Robert'}
Observation: 292123
> Running step 2aa00108-c5cb-453d-adf5-2ad4182edfd2. Step input: None
Thought: I have the order id. Now I can use another tool to estimate the delivery date.
Action: estimate_delivery_date_of_order
Action Input: {'order_id': '292123'}
Observation: Friday
> Running step 2d7fcba3-cf01-4e99-a61e-4316fd62edef. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Your order will arrive on Friday.


'Your order will arrive on Friday.'

In [5]:
agent = ReActAgent.from_tools([FunctionTool.from_defaults(fn=t) for t in tools], llm=llm)

response = agent.chat("My Name is Alice. I ordered something. And I would like to know when it will arrive.")
response.response

'Your order will arrive on Thursday.'